# Getting Started with Fine-Tuning Mistral 7B

This notebook shows you a simple example of how to LoRA finetune Mistral 7B. You can run this notebook in Google Colab with Pro + account with A100 and 40GB RAM.

<a target="_blank" href="https://colab.research.google.com/github/mistralai/mistral-finetune/blob/main/tutorials/mistral_finetune_7b.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Check out `mistral-finetune` Github repo to learn more: https://github.com/mistralai/mistral-finetune/

## Installation

Clone the `mistral-finetune` repo:


In [1]:
%cd /content/
!git clone https://github.com/mistralai/mistral-finetune.git

/content
Cloning into 'mistral-finetune'...
remote: Enumerating objects: 472, done.
remote: Counting objects: 100% (251/251), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 472 (delta 211), reused 161 (delta 159), pack-reused 221 (from 1)
Receiving objects: 100% (472/472), 243.32 KiB | 1.28 MiB/s, done.
Resolving deltas: 100% (251/251), done.


Install all required dependencies:

In [2]:
!pip install -r /content/mistral-finetune/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
 

## Model download

In [3]:
!pip install huggingface_hub

In [7]:
# huggingface login
from huggingface_hub import notebook_login

notebook_login()

In [8]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

! cp -r /root/mistral_models/7B-v0.3 /content/mistral_models
! rm -r /root/mistral_models/7B-v0.3

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

LocalEntryNotFoundError: An error happened while trying to locate the file on the Hub and we cannot find the requested files in the local cache. Please check your connection and try again or make sure your Internet connection is on.

In [9]:
# Alternatively, you can download the model from mistral

!wget https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar

--2024-10-27 15:19:49--  https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar
Resolving models.mistralcdn.com (models.mistralcdn.com)... 104.26.7.117, 172.67.70.68, 104.26.6.117, ...
Connecting to models.mistralcdn.com (models.mistralcdn.com)|104.26.7.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14496675840 (14G) [application/x-tar]
Saving to: ‘mistral-7B-v0.3.tar’

mistral-7B-v0.3.tar 100%[===================>]  13.50G  23.4MB/s    in 8m 33s  

2024-10-27 15:28:23 (26.9 MB/s) - ‘mistral-7B-v0.3.tar’ saved [14496675840/14496675840]



In [10]:
!DIR=/content/mistral_models && mkdir -p $DIR && tar -xf mistral-7B-v0.3.tar -C $DIR

In [11]:
!ls /content/mistral_models

consolidated.safetensors  params.json  tokenizer.model.v3


## Prepare dataset

To ensure effective training, mistral-finetune has strict requirements for how the training data has to be formatted. Check out the required data formatting [here](https://github.com/mistralai/mistral-finetune/tree/main?tab=readme-ov-file#prepare-dataset).

In this example, let’s use the ultrachat_200k dataset. We load a chunk of the data into Pandas Dataframes, split the data into training and validation, and save the data into the required `jsonl` format for fine-tuning.

In [12]:
%cd /content/

/content


In [13]:
# make a new directory called data
!mkdir -p data

In [14]:
# navigate to this data directory
%cd /content/data

/content/data


In [15]:
# read data into a pandas dataframe
import pandas as pd

df = pd.read_parquet('https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/main/data/test_gen-00000-of-00001-3d4cd8309148a71f.parquet')

In [16]:
# split data into training and evaluation
df_train=df.sample(frac=0.95,random_state=200)
df_eval=df.drop(df_train.index)

In [23]:
list = df_train.iloc[0].to_list()
list[2]

array([{'content': "Read the passage below and answer the question or follow the instruction: What are the areas of responsibilities of the new hires in West Virginia Power's front office staff including Nathan Richard, Hannah Frenchick, and Lindsey Webb? Passage: We have personnel news to report from the West Virginia Power (Low A; Sally League), Pittsfield Suns (summer collegiate; Futures League), and the Victoria HarbourCats (summer collegiate; West Coast League). The Power have announced several front office moves, highlighted by the elevation of Jeremy Taylor to General Manager. Two former interns received full-time offers this past offseason, while several others added on to their organizational responsibilities and Dean Thomas joined the club as their new Game Entertainment and Production Manager. Jesus Paez joins the full-time staff as the team’s Stadium Operations Manager, while Lindsey Webb comes on as the Suites and Community Outreach Coordinator. Meanwhile, Nathan Richard h

In [24]:
# save data into .jsonl files
df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)

In [26]:
!ls /content/data

ultrachat_chunk_eval.jsonl  ultrachat_chunk_train.jsonl


In [27]:
# navigate to the mistral-finetune directory
%cd /content/mistral-finetune/

/content/mistral-finetune


In [28]:
# some of the training data doesn't have the right format,
# so we need to reformat the data into the correct format and skip the cases that don't have the right format:

!python -m utils.reformat_data /content/data/ultrachat_chunk_train.jsonl

Skipped 3674th sample
Skipped 9176th sample
Skipped 10559th sample
Skipped 13293th sample
Skipped 13973th sample
Skipped 15219th sample


In [29]:
# eval data looks all good
!python -m utils.reformat_data /content/data/ultrachat_chunk_eval.jsonl

In [32]:
# Now you can verify your training yaml to make sure the data is correctly formatted and to get an estimate of your training time.

!python -m utils.validate_data --train_yaml example/7B.yaml


Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/content/mistral-finetune/utils/validate_data.py", line 372, in <module>
    main(args)
  File "/content/mistral-finetune/utils/validate_data.py", line 179, in main
    datasets, weights = parse_data_sources(pretrain_file, instruct_file)
  File "/content/mistral-finetune/finetune/data/dataset.py", line 159, in parse_data_sources
    assert min(n_weights) > 0
ValueError: min() arg is an empty sequence


## Start training

In [33]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [1]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
# data
data:
  instruct_data: "/content/data/ultrachat_chunk_train.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "/content/data/ultrachat_chunk_eval.jsonl"  # Optionally fill

# model
model_id_or_path: "/content/mistral_models"  # Change to downloaded path
lora:
  rank: 64

# optim
# tokens per training steps = batch_size x num_GPUs x seq_len
# we recommend sequence length of 32768
# If you run into memory error, you can try reduce the sequence length
seq_len: 4096
batch_size: 1
num_microbatches: 8
max_steps: 100
optim:
  lr: 1.e-4
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "/content/test_ultra"  # Fill
"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [2]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/test_ultra file
# ! rm -r /content/test_ultra

In [3]:
# start training

!torchrun --nproc-per-node 1 -m train example.yaml

/usr/bin/python3: No module named train
E1027 17:02:51.812000 3231 torch/distributed/elastic/multiprocessing/api.py:869] failed (exitcode: 1) local_rank: 0 (pid: 3246) of binary: /usr/bin/python3
Traceback (most recent call last):
  File "/usr/local/bin/torchrun", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 355, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/run.py", line 919, in main
    run(args)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/run.py", line 910, in run
    elastic_launch(
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/launcher/api.py", line 138, in __call__
    return launch_agent(self._config, self._entrypoint, list(args))
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/launcher/api.py", line 269, in launch_agent
    raise ChildFailedError(
t

## Inference

In [ ]:
!pip install mistral_inference

In [ ]:
from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file("/content/mistral_models/tokenizer.model.v3")  # change to extracted tokenizer file
model = Transformer.from_folder("/content/mistral_models")  # change to extracted model dir
model.load_lora("/content/test_ultra/checkpoints/checkpoint_000100/consolidated/lora.safetensors")

completion_request = ChatCompletionRequest(messages=[UserMessage(content="Explain Machine Learning to me in a nutshell.")])

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)

Machine learning is a subset of artificial intelligence that involves the use of algorithms to learn from data and make predictions or decisions without being explicitly programmed. It is a type of computer science that enables machines to learn and improve from experience without being explicitly programmed. Machine learning algorithms can learn from data and make predictions or decisions based
